# Modular CerberusTS Experimentation

In [1]:
import os
os.chdir("..")

In [2]:
from cerberus_ts import Cerberus, train_cerberus
from cerberus_ts import prepare_timeseries_data, generate_predictions
import pandas as pd

c:\Users\johnm\AppData\Local\Programs\Python\Python38\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
df = pd.read_csv(r"data/jena_climate_2009_2016.csv",
                parse_dates=['Date Time'], 
                index_col=['Date Time'])
df.index = pd.to_datetime(df.index, format='%d.%m.%Y %H:%M:%S')
df = df.iloc[:5000,:]

In [4]:
thresholds = {
    'call': 1,
    'response': 1,
    'context_0': 1,
    'context_1': 1,
    'context_2': 1
}

sizes = {
    'call': 24,
    'response': 8,
    'context_0': 24,
    'context_1': 12,
    'context_2': 6
}

window_timesteps = {
    'call': '10T',
    'response': '10T',
    'context_0': '1H',
    'context_1': '2H',
    'context_2': '6H'
}

window_sizes = {
    'call': 24,
    'response': 8,
    'context_0': 24,
    'context_1': 12,
    'context_2': 6
}

feature_indexes = {
    'call': range(0,14),
    'response': [0, 1, 4],
    'context_0': range(0,14),
    'context_1': range(0,14),
    'context_2': range(0,14)   
}

In [5]:
prepared_dataloaders, sliced_data = prepare_timeseries_data(df,
                        sizes,
                        thresholds,
                        feature_indexes,
                        window_timesteps,
                        train_len = 20_000,
                        feature_range = [0, 1])

In [6]:
model = Cerberus(sizes, feature_indexes)

In [7]:
model = train_cerberus(model, prepared_dataloaders, num_epochs = 30)

Epoch [1/30], Loss: 0.004035187437354277
Epoch [2/30], Loss: 0.0013750345267665883
Epoch [3/30], Loss: 0.001190887064440176
Epoch [4/30], Loss: 0.001123394260648638
Epoch [5/30], Loss: 0.0010858420922886581
Epoch [6/30], Loss: 0.0010633071213184545
Epoch [7/30], Loss: 0.001007985788358686
Epoch [8/30], Loss: 0.000959870748532315
Epoch [9/30], Loss: 0.0009263828008746108
Epoch [10/30], Loss: 0.000902067106605197
Epoch [11/30], Loss: 0.0008722367838102704
Epoch [12/30], Loss: 0.0008459301694529131
Epoch [13/30], Loss: 0.0008365164770899961
Epoch [14/30], Loss: 0.0008150792129648229
Epoch [15/30], Loss: 0.0007978582347277552
Epoch [16/30], Loss: 0.0007860557214977841
Epoch [17/30], Loss: 0.000775376315965938
Epoch [18/30], Loss: 0.0007526469927203531
Epoch [19/30], Loss: 0.0007412809496357417
Epoch [20/30], Loss: 0.0007297823271558931
Epoch [21/30], Loss: 0.0007096792297670618
Epoch [22/30], Loss: 0.0006933501824581375
Epoch [23/30], Loss: 0.0006873409432591871
Epoch [24/30], Loss: 0.0006

In [9]:
sel_indx = 4000
# Move model over to cpu
model.to("cpu")

# Set the model to evaluation mode
model.eval()
selected_data = {key: value[sel_indx:sel_indx+1,:] for key, value in sliced_data.items()}
responses_generated = generate_predictions(model,selected_data)